In [1]:
import os
import re
from pathlib import Path
from shutil import copyfile
import pandas as pd
import numpy as np

In [2]:
data_folder = './csv_data'
total_samples = 2400

In [3]:
df_count = pd.DataFrame(columns=['topic', 'count', 'proportion_count'])

csv_dfs = {}
total_count = 0

entries = os.listdir(data_folder)
regex = re.compile('data_([a-zA-Z]*)_final\.csv')

for filename in entries:
    if filename.startswith('.'): continue
    matched = regex.search(filename)
    if matched:
        topic_name = matched.group(1)
        csv_path = os.path.join(data_folder, filename)
        csv_dfs[topic_name] = pd.read_csv(csv_path, index_col=0)
        row_count = len(csv_dfs[topic_name].index)
        total_count += row_count
        df_count.loc[len(df_count)] = [topic_name, row_count, -1]
        print('{:<13}{:<7}'.format(topic_name, row_count))


Vaccine      2038   
AsianHate    1146   
Boomer       1712   
Political    22606  
Mask         13397  
Immigration  4998   


In [4]:
# df_count['proportion_count'] = df_count['count'].apply(lambda x: round((x/total_count)*total_samples))
df_count['proportion_count'] = round(total_samples / len(df_count.index))

In [5]:
df_count.to_csv('./csv_data/sample_count.csv', index=False)
df_count

,topic,count,proportion_count
0,Vaccine,2038,400
1,AsianHate,1146,400
2,Boomer,1712,400
3,Political,22606,400
4,Mask,13397,400
5,Immigration,4998,400


In [6]:
df_count['proportion_count'].sum()

2400

In [7]:
df_samples = pd.DataFrame(columns=['topic', 'hashtag', 'tweet_id', 'image_path', 'sample_path', 'body_text', 'image_text'])

for _, row in df_count.iterrows():
    topic_name = row['topic']
    df_topic = csv_dfs[topic_name].sample(n=row['proportion_count'], random_state=42)
    df_topic.insert(loc=0, column='topic', value=topic_name)
    df_topic.insert(loc=4, column='sample_path', value=np.nan)
    df_topic.insert(loc=len(df_topic.columns), column='image_text', value=np.nan)
    df_samples = df_samples.append(df_topic)

df_samples = df_samples.sample(frac=1, random_state=42).reset_index(drop=True)
df_samples.to_csv('./csv_data/sample_data.csv', index=False)
df_samples

,topic,hashtag,tweet_id,image_path,sample_path,body_text,image_text,text_with_OCR
0,Immigration,refugees,1297606828461883394,./data_Immigration/refugees/tweets4/EgIGg6aXsA...,NaN,Just one week left to get one of these hand em...,NaN,oe
1,Mask,maskup,1297309098669510656,./data_Mask/maskup/tweets4/EgD3vYTUcAE_JMC.jpg,NaN,🚧 #SFSO keeping our citizens updated on evacua...,NaN,er ha an ey st ge at Bra wy yo ae ae ay at oes...
2,Boomer,caresact,1291464418715459589,./data_Boomer/caresact/tweets1/Eew0B6PWAAYZPCX...,NaN,"Arguably, this one segment from the #CARESAct ...",NaN,aS is he
3,Mask,NoMasks,1290561571312328706,./data_Mask/NoMasks/tweets1/Eej-523WsAAJa6h.jpg,NaN,@RupertaMargate @FrancaisFarage @John35542712 ...,NaN,Ser ah
4,Immigration,cdnpoli,1297693936782213123,./data_Immigration/cdnpoli/tweets4/EgJVvpkUMAA...,NaN,"You beat me to it, lol .. #cpc #cpcldr #cdnpol...",NaN,You organize two car funeral procession iS
...,...,...,...,...,...,...,...,...
2395,Mask,NoMasks,1291460379638484994,./data_Mask/NoMasks/tweets1/EewwXv5WkAEb91v.jpg,NaN,What's this...a new wave of virus???\n\n#KBF #...,NaN,Let me introduce you to our next problem Seen ...
2396,Boomer,trumpliesamericansdie,1290437319611342851,./data_Boomer/trumpliesamericansdie/tweets1/Ee...,NaN,@realDonaldTrump 1st-To be so proud of a large...,NaN,reed la HI WA ha ant aa Trump to draw out at l...
2397,Boomer,caresact,1293904775055052801,./data_Boomer/caresact/tweets2/EfTfiMrWsAA35Y4...,NaN,Ready to re-open? Use your #CARESAct Funding t...,NaN,YOUR GO HERE Oe WEVE GOT vol
2398,Political,Trump2020,1291952240534728704,./data_Political/Trump2020/tweets2/Ee3vtmRX0AA...,NaN,@HowDoYouMakeAU2 @Jenny06980536 @TerriSm600279...,NaN,al LEAVE NEVER LEAVE


# Please only run cells below

## Copy selected samples and generate sample path

### Before running this part, please check that the topics you want to process is NOT in the annotation_data folder

In [1]:
import os
from pathlib import Path
from shutil import copyfile
import pandas as pd
import numpy as np

In [2]:
df_count = pd.read_csv('./csv_data/sample_count.csv')
df_samples = pd.read_csv('./csv_data/sample_data.csv')

In [3]:
def get_sample_path_and_copy(df, topic, annot_data_path):
    # if the sample_path already exists
    if not pd.isna(df['sample_path']):
        return df['sample_path']

    # if topic is not the selected topic
    if pd.isna(df['sample_path']) and df['topic'] != topic:
        return np.nan
    
    image_name = df['image_path'].split('/')[-1]
    dst_path = os.path.join(annot_data_path, image_name)
    copyfile(df['image_path'], dst_path)
    return dst_path

In [4]:
sample_folder = './annotation_data'
Path(sample_folder).mkdir(exist_ok=True)

for _, row in df_count.iterrows():
    topic_name = row['topic']
    topic_data_path = os.path.join('./', 'data_' + topic_name)
    annot_data_path = os.path.join(sample_folder, topic_name)
    # if data_{topic_name} folder doesn't exist
    if not Path(topic_data_path).is_dir():
        print('{} not exists; pass this topic'.format(topic_data_path))
        continue

    # if the sampled data for this topic already exists
    if Path(annot_data_path).is_dir():
        print('{} already exist; pass this topic'.format(annot_data_path))
        continue

    Path(annot_data_path).mkdir()

    df_samples['sample_path'] = df_samples.apply(get_sample_path_and_copy, axis=1, topic=topic_name, annot_data_path=annot_data_path)

    print('Topic \"{}\" processed'.format(topic_name))

Topic "Vaccine" processed
./data_AsianHate not exists; pass this topic
./data_Boomer not exists; pass this topic
./data_Political not exists; pass this topic
./data_Mask not exists; pass this topic
Topic "Immigration" processed


In [5]:
df_samples.to_csv('./csv_data/sample_data.csv', index=False)
df_samples

,topic,hashtag,tweet_id,image_path,sample_path,body_text,image_text,text_with_OCR
0,Immigration,refugees,1297606828461883394,./data_Immigration/refugees/tweets4/EgIGg6aXsA...,./annotation_data/Immigration/EgIGg6aXsAAX5-Y.jpg,Just one week left to get one of these hand em...,NaN,oe
1,Mask,maskup,1297309098669510656,./data_Mask/maskup/tweets4/EgD3vYTUcAE_JMC.jpg,NaN,🚧 #SFSO keeping our citizens updated on evacua...,NaN,er ha an ey st ge at Bra wy yo ae ae ay at oes...
2,Boomer,caresact,1291464418715459589,./data_Boomer/caresact/tweets1/Eew0B6PWAAYZPCX...,NaN,"Arguably, this one segment from the #CARESAct ...",NaN,aS is he
3,Mask,NoMasks,1290561571312328706,./data_Mask/NoMasks/tweets1/Eej-523WsAAJa6h.jpg,NaN,@RupertaMargate @FrancaisFarage @John35542712 ...,NaN,Ser ah
4,Immigration,cdnpoli,1297693936782213123,./data_Immigration/cdnpoli/tweets4/EgJVvpkUMAA...,./annotation_data/Immigration/EgJVvpkUMAALB8Z.png,"You beat me to it, lol .. #cpc #cpcldr #cdnpol...",NaN,You organize two car funeral procession iS
...,...,...,...,...,...,...,...,...
2395,Mask,NoMasks,1291460379638484994,./data_Mask/NoMasks/tweets1/EewwXv5WkAEb91v.jpg,NaN,What's this...a new wave of virus???\n\n#KBF #...,NaN,Let me introduce you to our next problem Seen ...
2396,Boomer,trumpliesamericansdie,1290437319611342851,./data_Boomer/trumpliesamericansdie/tweets1/Ee...,NaN,@realDonaldTrump 1st-To be so proud of a large...,NaN,reed la HI WA ha ant aa Trump to draw out at l...
2397,Boomer,caresact,1293904775055052801,./data_Boomer/caresact/tweets2/EfTfiMrWsAA35Y4...,NaN,Ready to re-open? Use your #CARESAct Funding t...,NaN,YOUR GO HERE Oe WEVE GOT vol
2398,Political,Trump2020,1291952240534728704,./data_Political/Trump2020/tweets2/Ee3vtmRX0AA...,NaN,@HowDoYouMakeAU2 @Jenny06980536 @TerriSm600279...,NaN,al LEAVE NEVER LEAVE
